**BTC Prediction**

In [1]:
!pip install pystan==2.19.1.1 --quiet
!pip install fbprophet --quiet
!pip install yfinance --quiet
!pip install Nasdaq-Data-Link

     |████████████████████████████████| 62.6 MB 1.2 MB/s 
     |████████████████████████████████| 64 kB 1.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1.1 requires cmdstanpy>=1.0.4, but you have cmdstanpy 0.9.5 which is incompatible.
     |████████████████████████████████| 62 kB 1.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

In [3]:
import nasdaqdatalink
btc_df = nasdaqdatalink.get("BCHAIN/MKPRU", authtoken="ugpNyJHCrKx_PeswJSpx")

In [4]:
btc_df.tail()

,Value
Date,
2022-12-05,"$17,117.57"
2022-12-06,"$16,967.02"
2022-12-07,"$17,085.45"
2022-12-08,"$16,836.05"
2022-12-09,"$17,234.58"


In [5]:
btc_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5090 entries, 2009-01-02 to 2022-12-09
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Value   5090 non-null   float64
dtypes: float64(1)
memory usage: 79.5 KB


In [6]:
btc_df.isnull().sum()

Value    0
dtype: int64

In [7]:
btc_df.reset_index(inplace=True)
btc_df.columns

Index(['Date', 'Value'], dtype='object')

In [8]:
df = btc_df[["Date", "Value"]]

new_names = {
    "Date": "ds", 
    "Value": "y",
}

df.rename(columns=new_names, inplace=True)

In [9]:
df.tail()

,ds,y
5085,2022-12-05,"$17,117.57"
5086,2022-12-06,"$16,967.02"
5087,2022-12-07,"$17,085.45"
5088,2022-12-08,"$16,836.05"
5089,2022-12-09,"$17,234.58"


In [10]:
# plot the open price

x = df["ds"]
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Set title
fig.update_layout(
    title_text="Time series plot of Bitcoin Price",
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
m = Prophet(
    seasonality_mode="multiplicative" 
)

m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [12]:
future = m.make_future_dataframe(periods = 365)
future.tail()

,ds
5450,2023-12-05
5451,2023-12-06
5452,2023-12-07
5453,2023-12-08
5454,2023-12-09


In [13]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
5450,2023-12-05,"$48,574.68","$39,875.64","$56,783.52"
5451,2023-12-06,"$49,188.78","$40,805.06","$58,033.46"
5452,2023-12-07,"$50,093.26","$41,091.99","$59,125.38"
5453,2023-12-08,"$50,740.73","$42,150.90","$59,953.03"
5454,2023-12-09,"$52,053.58","$43,524.25","$60,633.51"


In [14]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

forecast[forecast['ds'] == next_day]['yhat'].item()

43001.71505256025

In [15]:
plot_plotly(m, forecast)

In [16]:
plot_components_plotly(m, forecast)